In [ ]:
# Note: After you run this cell, the training and test data will be available in
# the file browser. (Click the folder icon on the left to view it)
#
# If you don't see the data after the cell completes, click the refresh button
# in the file browser (folder icon with circular arrow)

# First, let's download and unzip the data
!echo "Downloading files..."
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training1.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training2.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test_classes.csv

!echo "Unzipping files..."
!unzip -q /content/training1.zip
!unzip -q /content/training2.zip
!unzip -q /content/test.zip

# Combine the two traning directories
!echo "Merging training data..."
!mkdir /content/training
!mv /content/training1/* /content/training
!mv /content/training2/* /content/training

# Cleanup
!echo "Cleaning up..."
!rmdir /content/training1
!rmdir /content/training2
!rm training1.zip
!rm training2.zip
!rm test.zip

!echo "Data ready."

Unzipping files...
Merging training data...
Cleaning up...
Data ready.


In [ ]:
# Import libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Create an image training dataset
from tensorflow.keras.preprocessing import image_dataset_from_directory

# We're using keras' image_dataset_from_directory method to load our image data.
# See (https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory) for details
#
# A couple of things to note:
# 1. We're specifying a number for the seed, so we'll always get the same shuffle and split of our images.
# 2. Class names are inferred automatically from the image subdirectory names.
# 3. We're splitting the training data into 80% training, 20% validation.


training_dir = '/content/training/'
test_dir = '/content/test'
image_size = (100, 100)

# Split up the training data images into training and validations sets
training_data = image_dataset_from_directory(training_dir, validation_split=.2, subset='training', seed=42, image_size=image_size)
validation_data = image_dataset_from_directory(training_dir, validation_split=.2, subset='validation', seed=42, image_size=image_size)
# test_data = image_dataset_from_directory(test_dir, seed=42, image_size=image_size)

Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 39209 files belonging to 43 classes.
Using 7841 files for validation.


In [ ]:
import matplotlib.pyplot as plt
# View 9 images and their class labels
plt.figure(figsize=(10, 10))
for images, labels in training_data.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(training_data.class_names[labels[i]])
    plt.axis("off")

In [ ]:
# normalize data:
from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

#this part makes it so the data can run better, by optimizing memory management or something
AUTOTUNE = tf.data.AUTOTUNE

training_data = training_data.cache().prefetch(buffer_size=AUTOTUNE)
validation_data = validation_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 43 #number of signs
model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  training_data,
  validation_data=validation_data,
  epochs=10
)

Epoch 1/10
981/981 [==============================] - 35s 35ms/step - loss: 1.8219 - accuracy: 0.4980 - val_loss: 0.2590 - val_accuracy: 0.9272
Epoch 2/10
981/981 [==============================] - 32s 33ms/step - loss: 0.1836 - accuracy: 0.9476 - val_loss: 0.0879 - val_accuracy: 0.9788
Epoch 3/10
981/981 [==============================] - 32s 33ms/step - loss: 0.0764 - accuracy: 0.9778 - val_loss: 0.1013 - val_accuracy: 0.9754
Epoch 4/10
981/981 [==============================] - 33s 33ms/step - loss: 0.0539 - accuracy: 0.9842 - val_loss: 0.1194 - val_accuracy: 0.9716
Epoch 5/10
981/981 [==============================] - 32s 33ms/step - loss: 0.0441 - accuracy: 0.9874 - val_loss: 0.0672 - val_accuracy: 0.9841
Epoch 6/10
981/981 [==============================] - 33s 33ms/step - loss: 0.0269 - accuracy: 0.9920 - val_loss: 0.0809 - val_accuracy: 0.9830
Epoch 7/10
981/981 [==============================] - 33s 33ms/step - loss: 0.0366 - accuracy: 0.9901 - val_loss: 0.1169 - val_accuracy:

**Testing predictions**

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import mnist

In [ ]:
# TEST HOW GOOD THE PREDICTIONS ARE
dat = pd.read_csv("test/test_classes.csv")
predict_name_list = []
per_conf_score_list = []

for i in range(len(dat)):
  filename = dat.Filename[i]

  img = keras.preprocessing.image.load_img(
      "test/" + filename, target_size=(100,100)
  )

  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  predict_name_list.append(np.argmax(score))
  per_conf_score_list.append((100 * np.max(score)))

In [ ]:
df = pd.read_csv('/content/test/test_classes.csv')
df['guess'] = predict_name_list
df['accurate'] = df['ClassId'] == df['guess']
sum(df.accurate == True) / len(df)

0.9429136975455266

In [ ]:
!pip install coremltools

In [ ]:
import tensorflow as tf
import coremltools as ct

tf_keras_model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ]
)

# Pass in `tf.keras.Model` to the Unified Conversion API
mlmodel = ct.convert(tf_keras_model)
mlmodel.save('model')

Translating MIL ==> MLModel Ops: 100%|██████████| 13/13 [00:00<00:00, 854.53 ops/s]
